# Competitive Pricing Analysis Visualization

This notebook provides comprehensive visualization and analysis of competitive pricing data across multiple e-commerce platforms.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import sys
import os

# Add src to path for imports
sys.path.append('src')

from competitive_pricing import CompetitivePricingEngine, compare_prices

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 1. Load and Explore Data

In [ ]:
# Initialize the pricing engine
pricing_engine = CompetitivePricingEngine()

# Load the data
data = pd.read_csv('data/competitive_pricing_sample.csv')
data['date'] = pd.to_datetime(data['date'])

print(f"Dataset shape: {data.shape}")
print(f"Date range: {data['date'].min()} to {data['date'].max()}")
print(f"Products: {data['product_name'].unique()}")

data.head()

## 2. Price Comparison Analysis

In [ ]:
# Get price comparison for a specific product
product_id = 'P001'  # Basmati Rice
comparison = pricing_engine.compare_prices(product_id)

if comparison:
    print(f"Product: {comparison.product_name}")
    print(f"Lowest Price: ₹{comparison.lowest_price:.2f} on {comparison.lowest_platform}")
    print(f"Highest Price: ₹{comparison.highest_price:.2f} on {comparison.highest_platform}")
    print(f"Savings: ₹{comparison.savings_amount:.2f} ({comparison.price_difference_percentage:.1f}%)")
    print(f"Recommendation: {comparison.recommendation}")
else:
    print("No comparison data available")

## 3. Current Price Comparison Visualization

In [ ]:
# Create price comparison chart for all products
products = data['product_id'].unique()
platforms = ['amazon', 'flipkart', 'jiomart', 'blinkit', 'zepto', 'dmart_ready']

# Get latest prices for all products
latest_date = data['date'].max()
latest_data = data[data['date'] == latest_date]

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, product_id in enumerate(products):
    if i >= len(axes):
        break
        
    product_data = latest_data[latest_data['product_id'] == product_id]
    if not product_data.empty:
        prices = []
        platform_names = []
        
        for platform in platforms:
            price = product_data[platform].iloc[0]
            if pd.notna(price) and price > 0:
                prices.append(price)
                platform_names.append(platform.replace('_', ' ').title())
        
        if prices:
            bars = axes[i].bar(platform_names, prices)
            
            # Highlight lowest and highest prices
            min_idx = prices.index(min(prices))
            max_idx = prices.index(max(prices))
            
            bars[min_idx].set_color('green')
            bars[max_idx].set_color('red')
            
            axes[i].set_title(f"{product_data['product_name'].iloc[0]}")
            axes[i].set_ylabel('Price (₹)')
            axes[i].tick_params(axis='x', rotation=45)
            
            # Add value labels on bars
            for j, (bar, price) in enumerate(zip(bars, prices)):
                axes[i].text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(prices)*0.01,
                           f'₹{price:.0f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.suptitle('Current Price Comparison Across Platforms', y=1.02, fontsize=16)
plt.show()

## 4. Price Trends Over Time

In [ ]:
# Plot price trends for each product
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, product_id in enumerate(products):
    if i >= len(axes):
        break
        
    product_data = data[data['product_id'] == product_id]
    product_name = product_data['product_name'].iloc[0]
    
    for platform in platforms:
        if platform in product_data.columns:
            axes[i].plot(product_data['date'], product_data[platform], 
                        marker='o', label=platform.replace('_', ' ').title(), linewidth=2)
    
    axes[i].set_title(f"{product_name} - Price Trends")
    axes[i].set_xlabel('Date')
    axes[i].set_ylabel('Price (₹)')
    axes[i].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[i].grid(True, alpha=0.3)
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.suptitle('Price Trends Over Time by Platform', y=1.02, fontsize=16)
plt.show()

## 5. Platform Performance Analysis

In [ ]:
# Calculate platform statistics
platform_stats = []

for platform in platforms:
    if platform in data.columns:
        platform_prices = data[platform].dropna()
        if len(platform_prices) > 0:
            stats = {
                'Platform': platform.replace('_', ' ').title(),
                'Average Price': platform_prices.mean(),
                'Median Price': platform_prices.median(),
                'Min Price': platform_prices.min(),
                'Max Price': platform_prices.max(),
                'Price Std': platform_prices.std()
            }
            platform_stats.append(stats)

stats_df = pd.DataFrame(platform_stats)
print("Platform Performance Summary:")
print(stats_df.round(2))

In [ ]:
# Visualize platform performance
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Average prices by platform
ax1.bar(stats_df['Platform'], stats_df['Average Price'], color='skyblue')
ax1.set_title('Average Prices by Platform')
ax1.set_ylabel('Average Price (₹)')
ax1.tick_params(axis='x', rotation=45)

# Price range (min-max) by platform
ax2.bar(stats_df['Platform'], stats_df['Max Price'] - stats_df['Min Price'], color='lightcoral')
ax2.set_title('Price Range by Platform')
ax2.set_ylabel('Price Range (₹)')
ax2.tick_params(axis='x', rotation=45)

# Price volatility (standard deviation)
ax3.bar(stats_df['Platform'], stats_df['Price Std'], color='lightgreen')
ax3.set_title('Price Volatility by Platform')
ax3.set_ylabel('Standard Deviation (₹)')
ax3.tick_params(axis='x', rotation=45)

# Box plot of all prices by platform
platform_data = []
platform_labels = []
for platform in platforms:
    if platform in data.columns:
        prices = data[platform].dropna()
        if len(prices) > 0:
            platform_data.append(prices)
            platform_labels.append(platform.replace('_', ' ').title())

ax4.boxplot(platform_data, labels=platform_labels)
ax4.set_title('Price Distribution by Platform')
ax4.set_ylabel('Price (₹)')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6. Best Deals Analysis

In [ ]:
# Get best deals
best_deals = pricing_engine.get_best_deals(10)

if best_deals:
    deals_df = pd.DataFrame(best_deals)
    
    print("Top Deals (Highest Savings):")
    print(deals_df[['product_name', 'lowest_platform', 'lowest_price', 
                   'highest_price', 'savings_amount', 'savings_percentage']].round(2))
    
    # Visualize best deals
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Savings amount
    ax1.barh(deals_df['product_name'], deals_df['savings_amount'], color='gold')
    ax1.set_title('Potential Savings by Product')
    ax1.set_xlabel('Savings Amount (₹)')
    
    # Savings percentage
    ax2.barh(deals_df['product_name'], deals_df['savings_percentage'], color='orange')
    ax2.set_title('Savings Percentage by Product')
    ax2.set_xlabel('Savings Percentage (%)')
    
    plt.tight_layout()
    plt.show()
else:
    print("No deals data available")

## 7. Interactive Price Comparison Function

In [ ]:
def interactive_price_comparison(product_id):
    """Interactive function to compare prices for any product."""
    comparison = pricing_engine.compare_prices(product_id)
    
    if not comparison:
        print(f"No data found for product ID: {product_id}")
        return
    
    print(f"\n{'='*50}")
    print(f"PRICE COMPARISON: {comparison.product_name}")
    print(f"{'='*50}")
    
    # Current prices table
    print("\nCurrent Prices:")
    print("-" * 30)
    for platform, price in comparison.current_prices.items():
        status = ""
        if platform == comparison.lowest_platform:
            status = " 🏆 BEST"
        elif platform == comparison.highest_platform:
            status = " 💸 HIGHEST"
        print(f"{platform:<15}: ₹{price:>8.2f}{status}")
    
    print(f"\n💡 {comparison.recommendation}")
    
    # Trends
    if comparison.trend_7_days:
        print("\n7-Day Price Trends:")
        print("-" * 25)
        for platform, trend in comparison.trend_7_days.items():
            trend_icon = "📈" if trend > 0 else "📉" if trend < 0 else "➡️"
            print(f"{platform:<15}: {trend:>+6.1f}% {trend_icon}")
    
    # Create visualization
    plt.figure(figsize=(12, 6))
    
    # Current prices bar chart
    plt.subplot(1, 2, 1)
    platforms = list(comparison.current_prices.keys())
    prices = list(comparison.current_prices.values())
    
    colors = ['green' if p == comparison.lowest_platform else 
              'red' if p == comparison.highest_platform else 'skyblue' 
              for p in platforms]
    
    bars = plt.bar(platforms, prices, color=colors)
    plt.title(f'Current Prices - {comparison.product_name}')
    plt.ylabel('Price (₹)')
    plt.xticks(rotation=45)
    
    # Add value labels
    for bar, price in zip(bars, prices):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(prices)*0.01,
                f'₹{price:.0f}', ha='center', va='bottom')
    
    # Trends chart
    if comparison.trend_7_days:
        plt.subplot(1, 2, 2)
        trend_platforms = list(comparison.trend_7_days.keys())
        trend_values = list(comparison.trend_7_days.values())
        
        colors = ['green' if t > 0 else 'red' if t < 0 else 'gray' for t in trend_values]
        
        plt.bar(trend_platforms, trend_values, color=colors)
        plt.title('7-Day Price Trends')
        plt.ylabel('Price Change (%)')
        plt.xticks(rotation=45)
        plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Example usage
print("Available products:")
products = pricing_engine.get_product_list()
for product in products:
    print(f"- {product['product_id']}: {product['product_name']}")

# Compare prices for Basmati Rice
interactive_price_comparison('P001')

## 8. Summary and Insights

In [ ]:
# Generate summary insights
summary = pricing_engine.get_platform_summary()

print("COMPETITIVE PRICING ANALYSIS SUMMARY")
print("=" * 40)
print(f"Total Products Analyzed: {summary.get('total_products', 0)}")
print(f"Platforms Covered: {len(summary.get('platforms', []))}")
print(f"Date Range: {summary.get('date_range', {}).get('start', 'N/A')} to {summary.get('date_range', {}).get('end', 'N/A')}")

print("\nPlatform Rankings (by average price):")
if 'platform_stats' in summary:
    platform_avg = [(platform, stats['avg_price']) 
                   for platform, stats in summary['platform_stats'].items()]
    platform_avg.sort(key=lambda x: x[1])
    
    for i, (platform, avg_price) in enumerate(platform_avg, 1):
        print(f"{i}. {platform}: ₹{avg_price:.2f} (avg)")

print("\nKey Insights:")
if best_deals:
    max_savings = max(deal['savings_amount'] for deal in best_deals)
    best_deal_product = next(deal for deal in best_deals if deal['savings_amount'] == max_savings)
    print(f"• Highest savings opportunity: ₹{max_savings:.2f} on {best_deal_product['product_name']}")
    
    avg_savings = sum(deal['savings_amount'] for deal in best_deals) / len(best_deals)
    print(f"• Average potential savings: ₹{avg_savings:.2f} per product")

print("\n📊 Analysis complete! Use the interactive functions above to explore specific products.")